## 1. Process   
win 的 spawn模式 只支持参数拷贝，不支持文件对象和线程锁参数传递

In [3]:
# name; getpid()
# 在.py中运行
import multiprocessing
import threading
import os
import time

def fun():
    time.sleep(3)

def task():
    print(os.getpid(), os.getppid()) #获取自己的id， 在子进程获取父进程id
    print(multiprocessing.current_process().name) # 输出名字

if __name__ == "__main__":
    print(os.getpid(), os.getppid()) # 主进程id
    p1 = multiprocessing.Process(target=task)

    for _ in range(10):
        t = threading.Thread(target=fun)
        t.start()

    print(len(threading.enumerate())) # 获取线程个数

    p1.name = "name" # 设置名字

    p1.start()
    p1.join()  # 等待子进程结束


3104 13388
16


## 2. Process Class  
类似于线程类  

In [1]:
import multiprocessing

class MyProcess(multiprocessing.Process):
    def run(self):
        print('执行此进程', self._args)

if __name__ == '__main__':
    p = MyProcess(args=('xxx',))
    p.start()
    print("继续执行...")

继续执行...


## 3. IPC

#### Queue  
一端放一端取；FIFO

```python
queue = multiprocessing.Queue() 
queue.put(i)
queue.get(i)
````

In [ ]:
import multiprocessing

def put_in_queue(data_queue):
    for i in range(10):
        data_queue.put(i)

if __name__ == '__main__':
    queue = multiprocessing.Queue() # 创建queue
    
    p = multiprocessing.Process(target=put_in_queue, args=(queue,))
    p.start()
    p.join()
    
    for _ in range(10):
        print(queue.get())

#### Pipe  
![pipe通信](./images/pipe.png)

In [ ]:
import multiprocessing
import time

def child_send_recv(pipe):
    time.sleep(1)
    
    pipe.send([1, 2, 3])
    data = pipe.recv()  # 子进程阻塞直到接收
    print("子进程接收" + data)
    time.sleep(2)
    
if __name__ == '__main__':
    parent_pipe, child_pipe = multiprocessing.Pipe()
    
    p = multiprocessing.Process(target=child_send_recv, args=(child_pipe,)) # 子进程传入child pipe
    p.start() 
    
    parent_recv_data = parent_pipe.recv() # 父进程阻塞直到接收
    print("Parent recv"+parent_recv_data)
    parent_pipe.send(666)

#### Method 1 Shared Memory 不常用

In [3]:
from multiprocessing import Process, Value, Array

def func(data_array):
    data_array[0] = 666
    
if __name__ == '__main__':    
    arr = Array('i', [1, 2, 3, 4, 5])  # 共享数组： 元素必须数组i：int；并且不能增加或减少
    
    p = Process(target=func, args=(arr,))
    p.start()
    p.join()
    
    print(arr[:])

[1, 2, 3, 4, 5]


#### Method 2 Manager 不常用

In [ ]:
from multiprocessing import Process, Manager

def modify(dt, ls):
    dt[1] = "1"
    dt[2] = "2"
    ls.append("3")
    
if __name__ == '__main__':
    with Manager() as manager:
        
        dct = manager.dict() # 创建共享dict和list
        lst = manager.list()
        
        p = Process(target=modify, args=(dct, lst))
        p.start()
        p.join()
        
        print(lst) 
        print(dict)

## 3. Process Lock

In [ ]:
import multiprocessing

lock = multiprocessing.RLock() # 进程锁

p = multiprocessing.Process(target=func, args=(lock,)) # 需要把进程锁作为参数传入
p.start()

## 4. 进程池

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor


def task(num):
    print("执行", num)
    time.sleep(2)
    return num


def done(num):
    print("done", num.result()) # done的参数为fun的return的掩码， 用.result()获取结果


if __name__ == '__main__':
    pool = ThreadPoolExecutor(4)  # 创建进程池
    for i in range(10):
        future_1 = pool.submit(task, i)  # 提交任务
        future_1.add_done_callback(done)  # 主进程在task 完成的那一刻自动执行 done；done的参数为fun的return的掩码

    pool.shutdown(True)  # 等待所有进程结束
    print(1)



#### 进程池加锁需要用 multiprocessing.Manager.RLock()

In [ ]:
import time
import multiprocessing
from concurrent.futures.process import ProcessPoolExecutor


def task(lock):
    print("开始")
    # lock.acquire()
    # lock.release()
    with lock:                     # 使用进程安全的锁
        # 假设文件中保存的内容就是一个值：10
        with open('f1.txt', mode='r', encoding='utf-8') as f:
            current_num = int(f.read())
        print("排队抢票了")
        time.sleep(1)             # 模拟抢票耗时
        current_num -= 1
        with open('f1.txt', mode='w', encoding='utf-8') as f:
            f.write(str(current_num))


if __name__ == '__main__':
    # 进程池
    pool = ProcessPoolExecutor()

    # 不能直接使用 multiprocessing.RLock()，需要通过 Manager 创建
    manager = multiprocessing.Manager()
    lock_object = manager.RLock()   # 进程间共享的锁

    for _ in range(10):
        pool.submit(task, lock_object) # 传入锁

    pool.shutdown(True)     # 关闭池子并等待所有任务完成
